Load of Data

In [7]:
import pandas as pd

# Define the paths to your datasets
sensor_data_path = r'telemetry.csv'
failure_data_path = r'failures.csv'

# Load the sensor data
sensor_data = pd.read_csv(sensor_data_path)

# Load the failure data
failure_data = pd.read_csv(failure_data_path)

# Convert datetime columns to datetime type for both datasets
sensor_data['datetime'] = pd.to_datetime(sensor_data['datetime'])
failure_data['datetime'] = pd.to_datetime(failure_data['datetime'])

In [8]:
sensor_data.set_index('datetime', inplace=True)

# Sort by datetime to ensure rolling windows work correctly
sensor_data.sort_index(inplace=True)

# Choose a window size, e.g., 24 hours
window_size = '24H'

# Create rolling features for each sensor
for column in ['volt', 'rotate', 'pressure', 'vibration']:
    sensor_data[f'{column}_mean'] = sensor_data.groupby('machineID')[column].transform(lambda x: x.rolling(window_size).mean())
    sensor_data[f'{column}_std'] = sensor_data.groupby('machineID')[column].transform(lambda x: x.rolling(window_size).std())

# For each standard deviation column, fill NaN values with the average of available standard deviations for that sensor
for column in ['volt_std', 'rotate_std', 'pressure_std', 'vibration_std']:
    # Calculate the average standard deviation for the column, excluding NaN values
    avg_std = sensor_data[column].mean()
    
    # Fill NaN values in the standard deviation column with this average standard deviation
    sensor_data[column].fillna(avg_std, inplace=True)
    
print(sensor_data[['machineID', 'volt', 'volt_mean', 'volt_std', 'rotate', 'rotate_mean', 'rotate_std', 'pressure', 'pressure_mean', 'pressure_std', 'vibration', 'vibration_mean', 'vibration_std']].head(100))

/tmp/ipykernel_5561/1283988652.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sensor_data[f'{column}_mean'] = sensor_data.groupby('machineID')[column].transform(lambda x: x.rolling(window_size).mean())
/tmp/ipykernel_5561/1283988652.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sensor_data[f'{column}_std'] = sensor_data.groupby('machineID')[column].transform(lambda x: x.rolling(window_size).std())
/tmp/ipykernel_5561/1283988652.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sensor_data[f'{column}_mean'] = sensor_data.groupby('machineID')[column].transform(lambda x: x.rolling(window_size).mean())
/tmp/ipykernel_5561/1283988652.py:12: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  sensor_data[f'{column}_std'] = sensor_data.groupby('machineID')[column].transf

                     machineID        volt   volt_mean   volt_std      rotate  \
datetime                                                                        
2015-01-01 06:00:00          1  176.217853  176.217853  14.916611  418.504078   
2015-01-01 06:00:00         53  183.084582  183.084582  14.916611  420.980061   
2015-01-01 06:00:00         99  168.596133  168.596133  14.916611  384.747105   
2015-01-01 06:00:00         12  171.404215  171.404215  14.916611  576.923563   
2015-01-01 06:00:00          6  136.878588  136.878588  14.916611  492.088420   
...                        ...         ...         ...        ...         ...   
2015-01-01 06:00:00         30  179.301175  179.301175  14.916611  441.944165   
2015-01-01 06:00:00          3  185.482043  185.482043  14.916611  461.211137   
2015-01-01 06:00:00         32  161.194392  161.194392  14.916611  367.942046   
2015-01-01 06:00:00         70  162.628154  162.628154  14.916611  429.216322   
2015-01-01 06:00:00         

/tmp/ipykernel_5561/1283988652.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sensor_data[column].fillna(avg_std, inplace=True)


In [9]:
# Initialize a column for labels
sensor_data['failure_within_48h'] = 0

# For each failure, mark the preceding 48 hours as positive examples
for index, row in failure_data.iterrows():
    start_time = row['datetime'] - pd.Timedelta(hours=48)
    end_time = row['datetime']
    machine_id = row['machineID']
    
    sensor_data.loc[(sensor_data.index > start_time) & (sensor_data.index <= end_time) & (sensor_data['machineID'] == machine_id), 'failure_within_48h'] = 1
    

In [10]:
# Split the data based on a date. For example, use the last 20% of the dates as the test set.
split_date = sensor_data.index.max() - pd.Timedelta(days=365 * 0.2)  # Adjust based on your dataset's date range

train_data = sensor_data[sensor_data.index < split_date]
test_data = sensor_data[sensor_data.index >= split_date]

In [11]:
import xgboost as xgb
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# Features and target variable
X_train = train_data.drop(['failure_within_48h', 'machineID'], axis=1)
y_train = train_data['failure_within_48h']
X_test = test_data.drop(['failure_within_48h', 'machineID'], axis=1)
y_test = test_data['failure_within_48h']

# Apply SMOTE to balance the training data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Since we've applied SMOTE, convert the resampled datasets to DMatrix for XGBoost
dtrain_resampled = xgb.DMatrix(X_train_resampled, label=y_train_resampled)
dtest = xgb.DMatrix(X_test, label=y_test)

In [14]:
# Define XGBoost parameters
params = {
    'max_depth': 6,  # Depth of each tree
    'eta': 0.3,  # Learning rate
    'objective': 'binary:logistic',  # Binary classification
    'eval_metric': 'auc',  # Evaluation metric
    'nthread': 4  # Number of cores to use
}

num_boost_round = 100

# Train the model on the resampled (balanced) dataset
bst_resampled = xgb.train(params, dtrain_resampled, num_boost_round, evals=[(dtest, 'test')], early_stopping_rounds=10)

# Predict the probabilities of failure
y_pred_proba_resampled = bst_resampled.predict(dtest)

# Convert probabilities to binary predictions using a threshold, e.g., 0.5
y_pred_resampled = [1 if x > 0.5 else 0 for x in y_pred_proba_resampled]

# Calculate and print accuracy, precision, recall, and F1 score
print("Accuracy:", accuracy_score(y_test, y_pred_resampled))
print("Precision:", precision_score(y_test, y_pred_resampled))
print("Recall:", recall_score(y_test, y_pred_resampled))
print("F1 Score:", f1_score(y_test, y_pred_resampled))

# Generate and print the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_resampled)
print("Confusion Matrix:")
print(conf_matrix)

[0]	test-auc:0.90512
[1]	test-auc:0.90958
[2]	test-auc:0.91607
[3]	test-auc:0.92073
[4]	test-auc:0.92363
[5]	test-auc:0.92402
[6]	test-auc:0.92461
[7]	test-auc:0.92539
[8]	test-auc:0.92606
[9]	test-auc:0.92578
[10]	test-auc:0.92623
[11]	test-auc:0.92571
[12]	test-auc:0.92558
[13]	test-auc:0.92573
[14]	test-auc:0.92534
[15]	test-auc:0.92475
[16]	test-auc:0.92543
[17]	test-auc:0.92512
[18]	test-auc:0.92496
[19]	test-auc:0.92507
[20]	test-auc:0.92473
Accuracy: 0.8736052481460354
Precision: 0.20445103857566765
Recall: 0.886031184696994
F1 Score: 0.33223832916428075
Confusion Matrix:
[[147631  21448]
 [   709   5512]]


In [15]:
from sklearn.metrics import precision_recall_curve
import numpy as np

# Generate the precision-recall curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred_proba_resampled)

# Concatenate a threshold of 0 at the beginning (useful for plotting)
thresholds = np.insert(thresholds, 0, 0)

# Calculate the differences to target precision and recall
target_precision = 0.8
target_recall_range = (0.3, 0.35)
precision_diff = np.abs(precision - target_precision)
recall_in_range = (recall >= target_recall_range[0]) & (recall <= target_recall_range[1])

# Find the index of the threshold that meets the criteria
# Prioritize meeting the precision criterion and recall being in the specified range
valid_indices = np.where(recall_in_range)[0]  # Indices where recall is in the desired range
closest_precision_idx = valid_indices[np.argmin(precision_diff[valid_indices])]  # Closest to target precision within recall range

# Best threshold based on criteria
best_threshold = thresholds[closest_precision_idx]
print(f"Best Threshold for Target Precision ~{target_precision*100}% and Recall between {target_recall_range[0]*100}% and {target_recall_range[1]*100}%: {best_threshold}")

# Apply this threshold to convert probabilities to binary predictions
# Precision is 0.28 and Recall is 0.30 for 0.9102 threshold => y_pred_adjusted = [1 if prob >= best_threshold else 0 for prob in y_pred_proba_resampled]
y_pred_threshold_080 = [1 if prob >= 0.90 else 0 for prob in y_pred_proba_resampled]

# Evaluate the model with the new threshold
accuracy_080 = accuracy_score(y_test, y_pred_threshold_080)
precision_080 = precision_score(y_test, y_pred_threshold_080)
recall_080 = recall_score(y_test, y_pred_threshold_080)
f1_score_080 = f1_score(y_test, y_pred_threshold_080)

# Print the updated metrics
print(f"Metrics with 0.80 Threshold:")
print(f"Accuracy: {accuracy_080:.4f}")
print(f"Precision: {precision_080:.4f}")
print(f"Recall: {recall_080:.4f}")
print(f"F1 Score: {f1_score_080:.4f}")

# Generate and print the confusion matrix
conf_matrix_080 = confusion_matrix(y_test, y_pred_threshold_080)
print("Confusion Matrix with 0.80 Threshold:")
print(conf_matrix_080)

Best Threshold for Target Precision ~80.0% and Recall between 30.0% and 35.0%: 0.9102581739425659
Metrics with 0.80 Threshold:
Accuracy: 0.9432
Precision: 0.2758
Recall: 0.3691
F1 Score: 0.3157
Confusion Matrix with 0.80 Threshold:
[[163049   6030]
 [  3925   2296]]
